In [1]:
##### TAKEN FROM https://beckernick.github.io/matrix-factorization-recommender/ ######

# Import Necessary Libraries
import pandas as pd
import numpy as np

ratings_df0 = pd.read_csv('ratings.csv', header = 0, delimiter=',')
movies_df0 = pd.read_csv('movies.csv', header = 0, delimiter=',')
# movies_df['MovieID'] = movies_df['MovieID'].apply(pd.to_numeric)

In [2]:
# Make a Smaller Sample Version of Ratings Info
ratings_df = ratings_df0.drop(ratings_df0.index[1001:20000263])
print(ratings_df)

      userId  movieId  rating   timestamp
0          1        2     3.5  1112486027
1          1       29     3.5  1112484676
2          1       32     3.5  1112484819
3          1       47     3.5  1112484727
4          1       50     3.5  1112484580
5          1      112     3.5  1094785740
6          1      151     4.0  1094785734
7          1      223     4.0  1112485573
8          1      253     4.0  1112484940
9          1      260     4.0  1112484826
10         1      293     4.0  1112484703
11         1      296     4.0  1112484767
12         1      318     4.0  1112484798
13         1      337     3.5  1094785709
14         1      367     3.5  1112485980
15         1      541     4.0  1112484603
16         1      589     3.5  1112485557
17         1      593     3.5  1112484661
18         1      653     3.0  1094785691
19         1      919     3.5  1094785621
20         1      924     3.5  1094785598
21         1     1009     3.5  1112486013
22         1     1036     4.0  111

In [3]:
# Make a Smaller Sample Version of Movies Info
movies_df = movies_df0 #.drop(movies_df0.index[1001:20000263])
print(movies_df)

       movieId                                              title  \
0            1                                   Toy Story (1995)   
1            2                                     Jumanji (1995)   
2            3                            Grumpier Old Men (1995)   
3            4                           Waiting to Exhale (1995)   
4            5                 Father of the Bride Part II (1995)   
5            6                                        Heat (1995)   
6            7                                     Sabrina (1995)   
7            8                                Tom and Huck (1995)   
8            9                                Sudden Death (1995)   
9           10                                   GoldenEye (1995)   
10          11                     American President, The (1995)   
11          12                 Dracula: Dead and Loving It (1995)   
12          13                                       Balto (1995)   
13          14                    

In [4]:
# Transform into a User-Item-Rating Matrix
R_df = ratings_df.pivot(index = ratings_df.columns[0], columns = movies_df.columns[0], values = ratings_df.columns[0]).fillna(0)
print(R_df)

movieId  1      2      3      6      7      10     11     15     16     17     \
userId                                                                          
1          0.0    1.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2          0.0    0.0    2.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3          3.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4          0.0    0.0    0.0    4.0    0.0    4.0    0.0    0.0    0.0    0.0   
5          0.0    5.0    0.0    0.0    0.0    0.0    5.0    0.0    0.0    5.0   
6          6.0    0.0    6.0    0.0    6.0    0.0    0.0    0.0    0.0    6.0   
7          0.0    0.0    7.0    0.0    7.0    0.0    7.0    7.0    7.0    7.0   
8          8.0    0.0    8.0    8.0    0.0    8.0    0.0    0.0    0.0    0.0   
9          0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
10        10.0    0.0    0.0    0.0    0.0    0.0   10.0    0.0    0.0    0.0   
11        11.0    0.0    0.0

In [5]:
# Normalize Data by the Mean
R = R_df.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
R_demeaned = R - user_ratings_mean.reshape(-1, 1)

In [38]:
# Singular Value Decomposition
from scipy.sparse.linalg import svds
U, sigma, Vt = svds(R_demeaned, k = 10)

In [39]:
sigma = np.diag(sigma)

In [40]:
# Add User Means for Making Predictions from Decomposed Matrices
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)
preds_df = pd.DataFrame(all_user_predicted_ratings, columns = R_df.columns)

In [48]:
def recommend_movies(predictions_df, userID, movies_df, original_ratings_df, num_recommendations=5):
    
    # Get and sort the user's predictions
    user_row_number = userID - 1 # UserID starts at 1, not 0
    sorted_user_predictions = predictions_df.iloc[user_row_number].sort_values(ascending=False)
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings_df[original_ratings_df.UserID == (userID)]
    user_full = (user_data.merge(movies_df, how = 'left', left_on = 'MovieID', right_on = 'MovieID').
                     sort_values(['Rating'], ascending=False)
                 )

    print ('User {0} has already rated {1} movies.'.format(userID, user_full.shape[0]))
    print ('Recommending the highest {0} predicted ratings movies not already rated.'.format(num_recommendations))
    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (movies_df[~movies_df['MovieID'].isin(user_full['MovieID'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'MovieID',
               right_on = 'MovieID').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

already_rated, predictions = recommend_movies(preds_df, 11, movies_df, ratings_df, 10)

AttributeError: 'DataFrame' object has no attribute 'UserID'